In [1]:
import os
os.chdir("../../")
import pandas as pd
import numpy as np
import seaborn as sns

from scripts.python.PdfParse import *
from scripts.python.ts_utils import *
from scripts.python.utils import *

## Wrangling

In [2]:
folder_path = os.getcwd() + "/data/tourism/palau/"
file_path = folder_path + os.listdir(folder_path)[0]

In [3]:
palau = pd.read_excel(file_path, sheet_name="TabMth")
palau = palau.iloc[0:-1].reset_index().drop("index", axis=1)
palau.columns = palau.iloc[0]
palau = palau.iloc[1:].T
palau = palau.reset_index()
palau.columns = palau.iloc[0]
palau = (palau.rename({"Country Group": "Date"}, axis=1)
         .iloc[1:]
         .reset_index()
         .drop("index", axis=1))

In [4]:
# Change the data format
palau["Date"] = pd.to_datetime(palau["Date"])
for col in palau.columns[~palau.columns.isin(["Date"])]:
    palau[col] = palau[col].astype(float)


palau.to_csv(folder_path + "intermediate/palau_monthly_visitor.csv",
             encoding="utf-8")
palau.head(5)

,Date,JAPAN,SOUTH KOREA,TAIWAN,CHINA,USA/CANADA,EUROPE,OTHERS,Total
0,2007-06-01,856.0,1291.0,3245.0,86.0,669.0,99.0,463.0,6709.0
1,2007-07-01,2119.0,1366.0,3269.0,33.0,653.0,144.0,437.0,8021.0
2,2007-08-01,3476.0,1354.0,3046.0,46.0,580.0,256.0,438.0,9196.0
3,2007-09-01,3022.0,910.0,2497.0,61.0,559.0,145.0,401.0,7595.0
4,2007-10-01,1807.0,1082.0,2298.0,49.0,774.0,390.0,395.0,6795.0


In [5]:
get_adf_df(palau.diff().dropna(), ["Total"])

,Test Statistic,p-value,# Lags Used,Number of Observations Used,Critical Value (1%),Critical Value (5%),Critical Value (10%)
Total,-3.403905,0.010826,11.0,171.0,-3.469181,-2.878595,-2.575863


In [6]:
from bokeh.palettes import Category20
from bokeh.plotting import figure, show, output_file
from bokeh.models import ColumnDataSource, HoverTool, Legend


output_file('palau_month_by_country.html')
p = figure(height=600, width=1000,
           title="Number of Passengers Per Month",
           x_axis_type="datetime",
           x_axis_label="Date",
           y_axis_label="Number of Passengers Per Month",
           tooltips=[("Country", "$name"),
                     ("Passenger per month", "@$name")])

countries = palau.columns[~palau.columns.isin(["Date", "Total"])].to_list()
p.add_layout(Legend(), 'right')
source = ColumnDataSource(palau)

for country, color in zip(countries, Category20[12]):
    # have to use different colnames for y-coords so tooltip can refer to @$name
    p.line('Date', country, source=source, name=country,
           legend_label=country, color=color)

p.legend.label_text_font_size = '9pt'
p.legend.click_policy = "mute"
p.legend.location = "top_left"

show(p)

## Palau Official Statistics

In [7]:
folder = os.getcwd() + "/data/tourism/"
flights = os.getcwd() + "/data/tourism/aviation_seats_flights_pic.xlsx"

In [8]:
df = pd.read_excel(flights)
df = (df[(df.Country == "Palau") & (df.Aircraft_type == "passenger")]
      .reset_index()
      .drop("index", axis=1)
      [["Date", "Seats_arrivals_intl", "Seats_arrivals_total", "Number_of_flights_intl", "Number_of_flights_total"]])

df["Date"] = pd.to_datetime(df["Date"])

range_df = pd.DataFrame(pd.date_range(
    start='2019-01-01', end='2022-10-16'), columns=["Date"])
df = range_df.merge(df, on="Date", how="left")
df["Month"], df["Year"] = df["Date"].dt.month, df["Date"].dt.year

In [9]:
palau_ma = (df.groupby(by=["Year", "Month"])
            .sum()[:-1]
            .reset_index())
palau_ma["Date"] = pd.date_range(start='2019-01-01', end="2022-09-01", freq="MS")

palau_19_22 = palau[palau.Date >=
                    "2019-01-01"].reset_index().drop("index", axis=1)

In [10]:
palau_avi_merged = palau_ma.merge(palau_19_22, how="left", on="Date")
palau_avi_merged.head(5)

,Year,Month,Seats_arrivals_intl,Seats_arrivals_total,Number_of_flights_intl,Number_of_flights_total,Date,JAPAN,SOUTH KOREA,TAIWAN,CHINA,USA/CANADA,EUROPE,OTHERS,Total
0,2019,1,13048.0,13048.0,75.0,75.0,2019-01-01,1953.0,1169.0,919.0,2072.0,626.0,310.0,280.0,7329.0
1,2019,2,13281.0,13281.0,74.0,74.0,2019-02-01,2055.0,1035.0,1092.0,4059.0,702.0,438.0,371.0,9752.0
2,2019,3,12870.0,12870.0,73.0,73.0,2019-03-01,2434.0,1090.0,1190.0,2549.0,826.0,483.0,461.0,9033.0
3,2019,4,10806.0,10806.0,64.0,64.0,2019-04-01,1756.0,808.0,1099.0,3182.0,777.0,368.0,415.0,8405.0
4,2019,5,11472.0,11472.0,67.0,67.0,2019-05-01,1288.0,1095.0,1393.0,2489.0,607.0,185.0,421.0,7478.0


In [20]:
palau_avi_merged[["Seats_arrivals_intl", "Total"]].describe()

,Seats_arrivals_intl,Total
count,45.000000,45.000000
mean,5090.488889,2769.533333
std,5341.217731,3530.913310
min,0.000000,1.000000
25%,656.000000,79.000000
50%,2285.000000,825.000000
75%,10964.000000,6174.000000
max,16166.000000,9789.000000


## EDA
### Correlation and cross-correlation

In [11]:
from scipy.stats import pearsonr

corr_seats, _ = pearsonr(palau_ma["Seats_arrivals_total"], palau_19_22["Total"])
corr_seat_intl, _ = pearsonr(palau_ma["Number_of_flights_intl"], palau_19_22["Total"])
print(f"Pearson Correlation between FlightRadar's Seats Arrival and Palau's census data is{corr_seats: .4f}.", "\n", 
    f"Pearson Correlation between FlightRadar's # Of Flights Arrival and Palau's census data is{corr_seat_intl: .4f}.")

Pearson Correlation between FlightRadar's Seats Arrival and Palau's census data is 0.9710. 
 Pearson Correlation between FlightRadar's # Of Flights Arrival and Palau's census data is 0.9699.


In [12]:
incl_cols = ["Seats_arrivals_total", "Seats_arrivals_intl", "Total"]
palau_cor = palau_avi_merged[incl_cols].corr()
palau_cor

,Seats_arrivals_total,Seats_arrivals_intl,Total
Seats_arrivals_total,1.000000,1.000000,0.970974
Seats_arrivals_intl,1.000000,1.000000,0.970974
Total,0.970974,0.970974,1.000000


In [13]:
palau_cc = cross_corr_df(palau_avi_merged, "Seats_arrivals_intl", "Total")
palau_cc.head(5)

,lag,cross_corr_coef
0,0,0.970974
1,1,0.923272
2,2,0.826792
3,3,0.708908
4,4,0.622178


### Stationarity

In [14]:
get_adf_df(palau_avi_merged, incl_cols)

,Test Statistic,p-value,# Lags Used,Number of Observations Used,Critical Value (1%),Critical Value (5%),Critical Value (10%)
Seats_arrivals_total,-2.011353,0.281617,2.0,42.0,-3.596636,-2.933297,-2.604991
Seats_arrivals_intl,-2.011353,0.281617,2.0,42.0,-3.596636,-2.933297,-2.604991
Total,-1.838074,0.361729,6.0,38.0,-3.615509,-2.941262,-2.609200


In [17]:
palau_diff = palau_avi_merged.diff().dropna()
palau_diff2 = palau_diff.diff().dropna()
get_adf_df(palau_diff, incl_cols)

,Test Statistic,p-value,# Lags Used,Number of Observations Used,Critical Value (1%),Critical Value (5%),Critical Value (10%)
Seats_arrivals_total,-5.476862,0.000002,0.0,43.0,-3.592504,-2.931550,-2.604066
Seats_arrivals_intl,-5.476862,0.000002,0.0,43.0,-3.592504,-2.931550,-2.604066
Total,-2.622492,0.088437,5.0,38.0,-3.615509,-2.941262,-2.609200


### Granger Causality

In [18]:
grangers_causation_matrix(palau_diff2, incl_cols, maxlag=13)

,Seats_arrivals_total_x,Seats_arrivals_intl_x,Total_x
Seats_arrivals_total_y,1.0,1.0,0.0
Seats_arrivals_intl_y,1.0,1.0,0.0
Total_y,0.0,0.0,1.0
